# CS498DL Assignment 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from kaggle_submission import output_submission_csv
from models.neural_net import NeuralNetwork
from utils.data_process import get_CIFAR10_data
from sklearn.utils import shuffle
from dataclasses import dataclass, field, asdict
from typing import List, Callable, Dict

import os
# import time
import uuid
try:
    import dill as pickle
except ImportError:
    import pickle
        
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)  # set default size of plots

try:
    from tqdm.notebook import tqdm
except ImportError:
    def tqdm(x):
        return x

# For auto-reloading external modules
# See http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Loading CIFAR-10
Now that you have implemented a neural network that passes gradient checks and works on toy data, you will test your network on the CIFAR-10 dataset.

In [ ]:
# You can change these numbers for experimentation
# For submission be sure they are set to the default values
TRAIN_IMAGES = 49000
VAL_IMAGES = 1000
TEST_IMAGES = 10000

data = get_CIFAR10_data(TRAIN_IMAGES, VAL_IMAGES, TEST_IMAGES)
X_train, y_train = data['X_train'], data['y_train']
X_val, y_val = data['X_val'], data['y_val']
X_test, y_test = data['X_test'], data['y_test']

In [ ]:
X_train.shape

In [ ]:
@dataclass
class Stats:
    '''Class for keeping track of an stats for a trained model.'''
    epochs : int
    train_loss : np.ndarray = field(init=False)
    train_accuracy : np.ndarray = field(init=False)
    val_accuracy : np.ndarray = field(init=False)
    weight_norms : np.ndarray = field(init=False)
    
    def __post_init__(self):
        self.train_loss = np.zeros(self.epochs)
        self.train_accuracy = np.zeros(self.epochs)
        self.val_accuracy = np.zeros(self.epochs)
        self.weight_norms = np.zeros(self.epochs)
        
@dataclass
class TrainingParams:
    '''Class for keeping track of model params.'''
    hidden_size : int
    epochs : int
    learning_rate : float
    learning_rate_decay : float
    regularization : float
    num_layers : int

    # to not be initialized
    hidden_sizes : List[int] = field(default_factory=list)
    num_classes : int = 10
    batch_size : int = 200
    input_size : int = 32 * 32 * 3
        
    def __post_init__(self):
        self.hidden_sizes = [self.hidden_size] * (self.num_layers - 1)
        
@dataclass
class TwoLayerTrainingParams(TrainingParams):
    num_layers : int = 2
        
@dataclass
class ThreeLayerTrainingParams(TrainingParams):
    num_layers : int = 3
        
class Algorithm:
    @dataclass
    class AlgorithmImpl:
        algorithm : str
        
    @dataclass
    class SGD(AlgorithmImpl):
        algorithm : str = "SGD"
            
    @dataclass
    class Adam(AlgorithmImpl):
        algorithm : str = "Adam"

In [ ]:
def train_model(params : TrainingParams, algorithm : Algorithm, verbose : bool = True):
    # Initialize a new neural network model
    net = NeuralNetwork(params.input_size, 
                        params.hidden_sizes, 
                        params.num_classes, 
                        params.num_layers)

    stats = Stats(epochs = params.epochs)

    f_idx = lambda x : x - 1
    def print_stats_at(epoch):
        if verbose:
            print("".join(('epoch : {}, '.format(epoch),
                  'training loss : {}, '.format(stats.train_loss[f_idx(epoch)]),
                  'training accuracy: {}, '.format(stats.train_accuracy[f_idx(epoch)]), 
                  'validation accuracy : {}'.format(stats.val_accuracy[f_idx(epoch)])))
                 )

    # For each epoch...
    learning_rate = params.learning_rate
    for epoch in tqdm(range(1, params.epochs + 1)):

        # Shuffle the dataset
        global X_train, y_train
        X_train, y_train = shuffle(X_train, y_train, random_state=None)

        # Training
        # For each mini-batch...
        for batch in range(TRAIN_IMAGES // params.batch_size):
            # Create a mini-batch of training data and labels
            start_idx = batch * params.batch_size
            stop_idx = start_idx + params.batch_size
            batch_slice = slice(start_idx, stop_idx, None)

            X_batch = X_train[batch_slice, :]
            y_batch = y_train[batch_slice]

            # Run the forward pass of the model to get a prediction and compute the accuracy
            scores = net.forward(X_batch)
            prediction = np.argmax(scores, axis=1)
            stats.train_accuracy[f_idx(epoch)] += (prediction == y_batch).sum()

            # Run the backward pass of the model to compute the loss, and update the weights
            stats.train_loss[f_idx(epoch)] += net.backward(y_batch, params.regularization)
            net.update(learning_rate, opt = algorithm.algorithm, epoch = epoch)

        # normalize at the end
        stats.train_accuracy[f_idx(epoch)] /= TRAIN_IMAGES

        # Validation
        # No need to run the backward pass here, just run the forward pass to compute accuracy
        val_scores = net.forward(X_val)
        val_prediction = np.argmax(val_scores, axis=1)
        # normalize to get ratio
        stats.val_accuracy[f_idx(epoch)] += (val_prediction == y_val).sum() / len(y_val)

        stats.weight_norms[f_idx(epoch)] = net.weight_norms()
        
        if not (epoch) % 5:
            print_stats_at(epoch)

        # Implement learning rate decay
        learning_rate *= params.learning_rate_decay

    # print at final time
    print_stats_at(epoch)
    
    return net, stats

def plot_stats(some_stats):
    try:
        import plotly.graph_objects as go
        from plotly.subplots import make_subplots

        # fig = go.Figure()
        fig = make_subplots(rows=3, 
                            cols=1, 
                            subplot_titles=("Weight norm history", "Loss history", "Accuracy history"))
        # l2 norm
        fig.add_trace(go.Scatter(y=some_stats.weight_norms,
                            mode='lines+markers', name="W norm"), row=1, col=1)
        fig.update_yaxes(title_text="W norm", row=1, col=1)

        # losses
        fig.add_trace(go.Scatter(y = some_stats.train_loss,
                            mode='lines+markers',
                            name='training_loss'), row=2, col=1)
        # Update xaxis properties
        fig.update_yaxes(title_text="Loss", row=2, col=1)

        # losses
        fig.add_trace(go.Scatter(y = some_stats.train_accuracy,
                            mode='lines+markers',
                            name='train_accuracy'), row=3, col=1)
        fig.add_trace(go.Scatter(y = some_stats.val_accuracy,
                            mode='lines+markers',
                            name='val_accuracy'), row=3, col=1)
        # Update xaxis properties
        fig.update_yaxes(title_text="Accuracy", row=3, col=1)
        fig.update_xaxes(title_text="epochs", row=3, col=1)

        fig.show(renderer="notebook+pdf")
        return fig
    
    except ImportError:
        # Plot the loss function and train / validation accuracies
        plt.subplot(3, 1, 1)
        plt.plot(some_stats.weight_norms)
        plt.title('Weight norm history')
        plt.xlabel('Iteration')
        plt.ylabel('Frobenius norm of weights')

        plt.subplot(3, 1, 2)
        plt.plot(some_stats.train_loss)
        plt.title('Loss history')
        plt.xlabel('Iteration')
        plt.ylabel('Loss')

        plt.subplot(3, 1, 3)
        plt.plot(some_stats.train_accuracy, label='train')
        plt.plot(some_stats.val_accuracy, label='val')
        plt.title('Classification accuracy history')
        plt.xlabel('Epoch')
        plt.ylabel('Classification accuracy')
        plt.legend()

        plt.tight_layout()
        plt.show()

def report_test_accuracy(trained_network : NeuralNetwork):
    global X_test, y_test
    test_scores = trained_network.forward(X_test)
    test_prediction = np.argmax(test_scores, axis=1)
    return test_prediction, (test_prediction == y_test).sum() / len(y_test)
        
def generate_report(trained_network : NeuralNetwork, 
                    training_stats : Stats, 
                    verbose : bool = True):
    test_prediction, acc = report_test_accuracy(trained_network)
    if verbose:
        print("Test accuracy obtained is ", acc)
        plot_stats(training_stats)
    return test_prediction, acc

class ModelCheckpoint:
    def __init__(self, params, algorithm_cls):       
        self.checkpoint_path = os.path.join(os.getcwd(), 
                                            type(params).__name__ + "_" + algorithm_cls.__name__) 
        os.makedirs(self.checkpoint_path, exist_ok=True)
    
    def save(self, nn, stats, clean_old = False):
        if clean_old:
            files = filter(lambda x: x.endswith('.pkl'), os.listdir(self.checkpoint_path))
            files = [os.path.join(self.checkpoint_path, f) for f in files] # add path to each file
            [os.remove(f) for f in files]
        
        file_name = str(uuid.uuid4())
        with open(os.path.join(self.checkpoint_path , file_name + '.pkl'), 'wb') as f:
            pickle.dump(nn, f)
        with open(os.path.join(self.checkpoint_path , file_name + '.stats_pkl'), 'wb') as f:
            pickle.dump(stats, f)
    
    def load_latest(self):
        files = filter(lambda x: x.endswith('.pkl'), os.listdir(self.checkpoint_path))
        files = [os.path.join(self.checkpoint_path, f) for f in files] # add path to each file
        files.sort(key=lambda x: os.path.getmtime(x))
        file_name = files[-1]
        with open(os.path.join(self.checkpoint_path , file_name), 'rb') as f:
            nn = pickle.load(f)

        stat_files = filter(lambda x: x.endswith('.stats_pkl'), os.listdir(self.checkpoint_path))
        stat_files = [os.path.join(self.checkpoint_path, f) for f in stat_files] # add path to each file
        stat_files.sort(key=lambda x: os.path.getmtime(x))
        stat_file_name = stat_files[-1]
    
        def file_name_without_ext(fn):
            temp = os.path.splitext(fn)
            var = os.path.basename(temp[0])
            return var
        
        assert file_name_without_ext(stat_file_name) == file_name_without_ext(file_name)
        
        with open(os.path.join(self.checkpoint_path , stat_file_name), 'rb') as f:
            stats = pickle.load(f)

        return nn, stats

In [ ]:
# Hyperparameters
input_size = 32 * 32 * 3

"""
# Given by the CS498 junta
num_layers = 2
hidden_size = 20
hidden_sizes = [hidden_size] * (num_layers - 1)
num_classes = 10
epochs = 100
batch_size = 200
learning_rate = 1e-3
learning_rate_decay = 0.95
regularization = 0.1
"""

optimization_algorithm = "SGD"
# optimization_algorithm = "Adam"

if optimization_algorithm == "SGD":
    """
    num_layers = 2
    hidden_size = 80, # even 100 is the same
    hidden_sizes = [hidden_size] * (num_layers - 1)
    num_classes = 10
    epochs = 30
    batch_size = 200
    learning_rate = 1e-2
    learning_rate_decay = 0.99
    regularization = 0.03
    # epoch : 29, training loss : 394.92802593721126, training accuracy: 0.5258163265306123, validation accuracy : 0.503

    num_layers = 2
    hidden_size = 100
    hidden_sizes = [hidden_size] * (num_layers - 1)
    num_classes = 10
    epochs = 30
    batch_size = 200
    learning_rate = 1e-2
    learning_rate_decay = 0.99
    regularization = 0.01    
    epoch : 30, training loss : 368.79332463881946, training accuracy: 0.5802857142857143, validation accuracy : 0.531
    """
    num_layers = 2
    hidden_size = 100
    hidden_sizes = [hidden_size] * (num_layers - 1)
    num_classes = 10
    epochs = 30
    batch_size = 200
    learning_rate = 1e-2
    learning_rate_decay = 0.99
    regularization = 0.01    
elif optimization_algorithm == "Adam":
    """
    num_layers = 2
    hidden_size = 80
    hidden_sizes = [hidden_size] * (num_layers - 1)
    num_classes = 10
    epochs = 20
    batch_size = 200
    learning_rate = 1e-2
    learning_rate_decay = 0.95
    regularization = 0.005
    epoch : 20, training loss : 345.92703000778187, training accuracy: 0.556795918367347, validation accuracy : 0.524


    num_layers = 2
    hidden_size = 100
    hidden_sizes = [hidden_size] * (num_layers - 1)
    num_classes = 10
    epochs = 20
    batch_size = 200
    learning_rate = 1e-2
    learning_rate_decay = 0.95
    regularization = 0.002
    epoch : 20, training loss : 311.3749559459592, training accuracy: 0.6064693877551021, validation accuracy : 0.54

    num_layers = 2
    hidden_size = 100
    hidden_sizes = [hidden_size] * (num_layers - 1)
    num_classes = 10
    epochs = 30
    batch_size = 200
    learning_rate = 1e-2
    learning_rate_decay = 0.95
    regularization = 0.005
    epoch : 30, training loss : 324.4904180432253, training accuracy: 0.5893061224489796, validation accuracy : 0.55    
    """
    num_layers = 2
    hidden_size = 100
    hidden_sizes = [hidden_size] * (num_layers - 1)
    num_classes = 10
    epochs = 20
    batch_size = 200
    learning_rate = 1e-2
    learning_rate_decay = 0.95
    regularization = 0.005

# Initialize a new neural network model
net = NeuralNetwork(input_size, hidden_sizes, num_classes, num_layers)

# Variables to store performance for each epoch
train_loss = np.zeros(epochs)
train_accuracy = np.zeros(epochs)
val_accuracy = np.zeros(epochs)

f_idx = lambda x : x - 1
def print_stats_at(epoch):
    print("".join(('epoch : {}, '.format(epoch),
          'training loss : {}, '.format(train_loss[f_idx(epoch)]),
          'training accuracy: {}, '.format(train_accuracy[f_idx(epoch)]), 
          'validation accuracy : {}'.format(val_accuracy[f_idx(epoch)])))
         )

# For each epoch...
for epoch in tqdm(range(1, epochs + 1)):
    
    # Shuffle the dataset
    X_train, y_train = shuffle(X_train, y_train, random_state=None)

    # Training
    # For each mini-batch...
    for batch in range(TRAIN_IMAGES // batch_size):
        # Create a mini-batch of training data and labels
        start_idx = batch * batch_size
        stop_idx = start_idx + batch_size
        batch_slice = slice(start_idx, stop_idx, None)

        X_batch = X_train[batch_slice, :]
        y_batch = y_train[batch_slice]
        # print(y_batch.shape)
        
        # Run the forward pass of the model to get a prediction and compute the accuracy
        scores = net.forward(X_batch)
        prediction = np.argmax(scores, axis=1)
        train_accuracy[f_idx(epoch)] += (prediction == y_batch).sum()

        # Run the backward pass of the model to compute the loss, and update the weights
        train_loss[f_idx(epoch)] += net.backward(y_batch, regularization)
        net.update(learning_rate, opt = optimization_algorithm, epoch = epoch)

    # normalize at the end
    train_accuracy[f_idx(epoch)] /= TRAIN_IMAGES
    
    # Validation
    # No need to run the backward pass here, just run the forward pass to compute accuracy
    val_scores = net.forward(X_val)
    val_prediction = np.argmax(val_scores, axis=1)
    # normalize to get ratio
    val_accuracy[f_idx(epoch)] += (val_prediction == y_val).sum() / len(y_val)
    
    if not (epoch) % 5:
        print_stats_at(epoch)
    
    # Implement learning rate decay
    learning_rate *= learning_rate_decay

# print at final time
print_stats_at(epoch)

## Graph loss and train/val accuracies

Examining the loss graph along with the train and val accuracy graphs should help you gain some intuition for the hyperparameters you should try in the hyperparameter tuning below. It should also help with debugging any issues you might have with your network.

In [ ]:
# Plot the loss function and train / validation accuracies
plt.subplot(2, 1, 1)
plt.plot(train_loss)
plt.title('Loss history')
plt.xlabel('Iteration')
plt.ylabel('Loss')

plt.subplot(2, 1, 2)
plt.plot(train_accuracy, label='train')
plt.plot(val_accuracy, label='val')
plt.title('Classification accuracy history')
plt.xlabel('Epoch')
plt.ylabel('Classification accuracy')
plt.legend()

plt.tight_layout()
plt.show()

## Results
We only show the relevant parts of our results (training reports and plots) and leave out the details of implementation. The implementation can be found in the attached notebook.  

## Train using SGD
To train our network we will use SGD. In addition, we will adjust the learning rate with an exponential learning rate schedule as optimization proceeds; after each epoch, we will reduce the learning rate by multiplying it by a decay rate.

You can try different numbers of layers and other hyperparameters on the CIFAR-10 dataset below.

### Two layer training with SGD

In [ ]:
""" Best Coarse search results, fine search results not included, see section on Hyperparameter tuning below
1
epoch : 30, training loss : 370.39517725296315, training accuracy: 0.5788775510204082, validation accuracy : 0.527

{'hidden_size': 100,
 'epochs': 30,
 'learning_rate': 0.01,
 'learning_rate_decay': 0.99,
 'regularization': 0.01,
 'num_layers': 2,
 'hidden_sizes': [100],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}
 
2
More or less saturated
epoch : 30, training loss : 366.56928722194294, training accuracy: 0.5678367346938775, validation accuracy : 0.543

{'hidden_size': 80,
 'epochs': 30,
 'learning_rate': 0.01,
 'learning_rate_decay': 0.99,
 'regularization': 0.01,
 'num_layers': 2,
 'hidden_sizes': [80],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}
 
3
Saturated
epoch : 30, training loss : 365.5431457113199, training accuracy: 0.5580408163265306, validation accuracy : 0.521

{'hidden_size': 60,
 'epochs': 30,
 'learning_rate': 0.01,
 'learning_rate_decay': 0.99,
 'regularization': 0.01,
 'num_layers': 2,
 'hidden_sizes': [60],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}

4
Saturates very quickly upon increasing learnign rate
epoch : 20, training loss : 340.3045509467915, training accuracy: 0.5846326530612245, validation accuracy : 0.514

{'hidden_size': 80,
 'epochs': 20,
 'learning_rate': 0.05,
 'learning_rate_decay': 0.99,
 'regularization': 0.01,
 'num_layers': 2,
 'hidden_sizes': [80],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}
 
5
For a low learning rate, it saturates late, but the training accuracy is somewhat lesser
epoch : 30, training loss : 407.2636209485998, training accuracy: 0.5327142857142857, validation accuracy : 0.506

{'hidden_size': 80,
 'epochs': 30,
 'learning_rate': 0.005,
 'learning_rate_decay': 0.99,
 'regularization': 0.01,
 'num_layers': 2,
 'hidden_sizes': [80],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}

6
For a slightly lower decay, things are more or less the same.
However for a devay rate of 0.9 things converge quickly and give bad results.
epoch : 30, training loss : 392.1978567049139, training accuracy: 0.5486734693877551, validation accuracy : 0.51

{'hidden_size': 80,
 'epochs': 30,
 'learning_rate': 0.01,
 'learning_rate_decay': 0.95,
 'regularization': 0.01,
 'num_layers': 2,
 'hidden_sizes': [80],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}

epoch : 30, training loss : 422.5223690800481, training accuracy: 0.5222653061224489, validation accuracy : 0.484

{'hidden_size': 80,
 'epochs': 30,
 'learning_rate': 0.01,
 'learning_rate_decay': 0.9,
 'regularization': 0.01,
 'num_layers': 2,
 'hidden_sizes': [80],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}
 
7
For lower values of regularization coefficient (0.001, 0.005), there is definitely overfitting 
epoch : 30, training loss : 307.4043522173789, training accuracy: 0.588, validation accuracy : 0.51

{'hidden_size': 80,
 'epochs': 30,
 'learning_rate': 0.01,
 'learning_rate_decay': 0.99,
 'regularization': 0.001,
 'num_layers': 2,
 'hidden_sizes': [80],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072} 
 
8
For higher reg (0.02, 0.04) things either take longer to converge with a moderate learnign rate
or converge at a slightly smaller accuracy
epoch : 30, training loss : 383.13177333741055, training accuracy: 0.5468571428571428, validation accuracy : 0.511

{'hidden_size': 80,
 'epochs': 30,
 'learning_rate': 0.01,
 'learning_rate_decay': 0.99,
 'regularization': 0.02,
 'num_layers': 2,
 'hidden_sizes': [80],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}
 
Most Optimal Found:
epoch : 30, training loss : 365.30765684235206, training accuracy: 0.5643469387755102, validation accuracy : 0.522

{'hidden_size': 80,
 'epochs': 30,
 'learning_rate': 0.02,
 'learning_rate_decay': 0.99,
 'regularization': 0.02,
 'num_layers': 2,
 'hidden_sizes': [80],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}
 
 epoch : 30, training loss : 367.225473479852, training accuracy: 0.5606326530612245, validation accuracy : 0.536

{'hidden_size': 80,
 'epochs': 30,
 'learning_rate': 0.05,
 'learning_rate_decay': 0.99,
 'regularization': 0.02,
 'num_layers': 2,
 'hidden_sizes': [80],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}

Based on these results, we vary parameters as 
hidden_size = [80, 100, 120]
epochs = 30
learning_rate = [1e-2, 2e-2, 5e-2, (8e-2)]
learning_rate_decay = [0.95, 0.99]
regularization = [0.005, 0.01, 0.02, (0.04)]
"""

# These are the best parameter sets found after running a fine search of the phase space
minimal_optimal_params = {
    "hidden_size": 120,
    "epochs": 30,
    "learning_rate": 8e-2,
    "learning_rate_decay": 0.95,
    "regularization": 0.01,
}

params = TwoLayerTrainingParams(**minimal_optimal_params)
if False:
    # Train a new model
    trained_net, stats = train_model(params, Algorithm.SGD)
else:
    # Load model from previously trained, use for final output and kaggle submission
    trained_net, stats = ModelCheckpoint(params, Algorithm.SGD).load_latest()
asdict(params)

In [ ]:
best_2layer_sgd_prediction, best_2layer_sgd_accuracy = generate_report(trained_net, stats)
if False:
    output_submission_csv('./nn_2layer_sgd_submission.csv', best_2layer_sgd_prediction)
    ModelCheckpoint(params, Algorithm.SGD).save(trained_net, stats, clean_old=True)

### Three layer training with SGD

In [ ]:
""" Best Coarse search results, fine search results not included, see section on Hyperparameter tuning below
 
Seems good
epoch : 30, training loss : 385.3684869757984, training accuracy: 0.5543673469387755, validation accuracy : 0.523

{'hidden_size': 100,
 'epochs': 30,
 'learning_rate': 0.05,
 'learning_rate_decay': 0.99,
 'regularization': 0.02,
 'num_layers': 3,
 'hidden_sizes': [100, 100],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}
 
A slightly higher learning rate but paired with a faster decay also helps
epoch : 30, training loss : 375.8179540220022, training accuracy: 0.5774081632653061, validation accuracy : 0.537

{'hidden_size': 100,
 'epochs': 30,
 'learning_rate': 0.08,
 'learning_rate_decay': 0.95,
 'regularization': 0.02,
 'num_layers': 3,
 'hidden_sizes': [100, 100],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}
 
 2. Starts overfitting like crazy for slighly smaller lambdas too
epoch : 30, training loss : 307.3058261118782, training accuracy: 0.6529591836734694, validation accuracy : 0.532

{'hidden_size': 100,
 'epochs': 30,
 'learning_rate': 0.05,
 'learning_rate_decay': 0.99,
 'regularization': 0.005,
 'num_layers': 3,
 'hidden_sizes': [100, 100],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}
 
 Meanwhile for higher lambdas, it doesnt budge much
 epoch : 30, training loss : 450.69301639983144, training accuracy: 0.47044897959183674, validation accuracy : 0.464

{'hidden_size': 100,
 'epochs': 30,
 'learning_rate': 0.05,
 'learning_rate_decay': 0.99,
 'regularization': 0.05,
 'num_layers': 3,
 'hidden_sizes': [100, 100],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}
 
 
We sweep the same parameters in this case.
"""
# These are the best parameter sets found after running a fine search of the phase space
# In this case, two parameter sets gave good, equivalent results
minimal_optimal_params_candidate_1 = {
    "hidden_size": 100,
    "epochs": 25,
    "learning_rate": 8e-2,
    "learning_rate_decay": 0.95,
    "regularization": 0.01,
}

minimal_optimal_params_candidate_2 = {
    "hidden_size": 120,
    "epochs": 30,
    "learning_rate": 5e-2,
    "learning_rate_decay": 0.95,
    "regularization": 0.01,
}

params = ThreeLayerTrainingParams(**minimal_optimal_params_candidate_2)

if False:
    # Train a new model
    trained_net, stats = train_model(params, Algorithm.SGD)
else:
    # Load model from previously trained, use for final output and kaggle submission
    trained_net, stats = ModelCheckpoint(params, Algorithm.SGD).load_latest()
asdict(params)

In [ ]:
best_3layer_sgd_prediction, best_3layer_sgd_accuracy = generate_report(trained_net, stats)
if False:
    output_submission_csv('./nn_3layer_sgd_submission.csv', best_3layer_sgd_prediction)
    ModelCheckpoint(params, Algorithm.SGD).save(trained_net, stats, clean_old=True)

## Train using Adam
Next we will train the same model using the Adam optimizer. You should take the above code for SGD and modify it to use Adam instead. For implementation details, see the lecture slides. The original paper that introduced Adam is also a good reference, and contains suggestions for default values: https://arxiv.org/pdf/1412.6980.pdf

### Two layer training with Adam

In [ ]:
""" Best Coarse search results, fine search results not included, see section on Hyperparameter tuning below
1.
weight norm has an interesting trend where it first drops and then increases back again

epoch : 30, training loss : 323.03653688232475, training accuracy: 0.5964489795918367, validation accuracy : 0.547

{'hidden_size': 100,
 'epochs': 30,
 'learning_rate': 0.01,
 'learning_rate_decay': 0.95,
 'regularization': 0.005,
 'num_layers': 2,
 'hidden_sizes': [100],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}
 
2.
Increasing regularization slightly worsens performane, but not alarmingly.
Decresing regularization overfits at the same level of performance
epoch : 30, training loss : 351.9264538779735, training accuracy: 0.5538775510204081, validation accuracy : 0.53

{'hidden_size': 100,
 'epochs': 30,
 'learning_rate': 0.01,
 'learning_rate_decay': 0.95,
 'regularization': 0.01,
 'num_layers': 2,
 'hidden_sizes': [100],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}
 
 epoch : 30, training loss : 282.7108224406282, training accuracy: 0.6563469387755102, validation accuracy : 0.539

{'hidden_size': 100,
 'epochs': 30,
 'learning_rate': 0.01,
 'learning_rate_decay': 0.95,
 'regularization': 0.002,
 'num_layers': 2,
 'hidden_sizes': [100],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}

3.
Didnt reach convergence yer, even with a higher learning rate
epoch : 30, training loss : 377.2422995167926, training accuracy: 0.5025714285714286, validation accuracy : 0.501

{'hidden_size': 100,
 'epochs': 30,
 'learning_rate': 0.02,
 'learning_rate_decay': 0.95,
 'regularization': 0.01,
 'num_layers': 2,
 'hidden_sizes': [100],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}

increased learning rate with increased decays also performs same more or less
epoch : 30, training loss : 313.03113190638817, training accuracy: 0.6172244897959184, validation accuracy : 0.541

{'hidden_size': 100,
 'epochs': 30,
 'learning_rate': 0.05,
 'learning_rate_decay': 0.9,
 'regularization': 0.002,
 'num_layers': 2,
 'hidden_sizes': [100],
 'num_classes': 10,
 'batch_size': 200,
 'input_size': 3072}

We sweep the same parameters as SGD in this case.
"""
    
# These are the best parameter sets found after running a fine search of the phase space
minimal_optimal_params = {
    "hidden_size": 100,
    "epochs": 30,
    "learning_rate": 1e-2,
    "learning_rate_decay": 0.95,
    "regularization": 0.005,
}
params = TwoLayerTrainingParams(**minimal_optimal_params)
if False:
    # Train a new model
    trained_net, stats = train_model(params, Algorithm.Adam)
else:
    # Load model from previously trained, use for final output and kaggle submission
    trained_net, stats = ModelCheckpoint(params, Algorithm.Adam).load_latest()
asdict(params)

In [ ]:
best_2layer_adam_prediction, best_2layer_adam_accuracy = generate_report(trained_net, stats)
if False:
    output_submission_csv('./nn_2layer_adam_submission.csv', best_2layer_adam_prediction)
    ModelCheckpoint(params, Algorithm.Adam).save(trained_net, stats, clean_old=True)

### Three layer training with Adam

In [ ]:
""" Best Coarse search results, fine search results not included, see section on Hyperparameter tuning below

We sweep the same parameters as SGD in this case.
"""
minimal_optimal_params_candidate_1 = {
    "hidden_size": 80,
    "epochs": 40,
    "learning_rate": 1e-2,
    "learning_rate_decay": 0.95,
    "regularization": 0.005,
}

minimal_optimal_params_candidate_2 = {
    "hidden_size": 100,
    "epochs": 40,
    "learning_rate": 1e-2,
    "learning_rate_decay": 0.95,
    "regularization": 0.005,
}

# These are the best parameter sets found after running a fine search of the phase space
params = ThreeLayerTrainingParams(**minimal_optimal_params_candidate_1)
if False:
    # Train a new model
    trained_net, stats = train_model(params, Algorithm.Adam)
else:
    # Load model from previously trained, use for final output and kaggle submission
    trained_net, stats = ModelCheckpoint(params, Algorithm.Adam).load_latest()
asdict(params)

In [ ]:
best_3layer_adam_prediction, best_3layer_adam_accuracy = generate_report(trained_net, stats)
if True:
    output_submission_csv('./nn_3layer_adam_submission.csv', best_3layer_adam_prediction)
    ModelCheckpoint(params, Algorithm.Adam).save(trained_net, stats, clean_old=True)

## Hyperparameter tuning

Once you have successfully trained a network you can tune your hyparameters to increase your accuracy.

Based on the graphs of the loss function above you should be able to develop some intuition about what hyperparameter adjustments may be necessary. A very noisy loss implies that the learning rate might be too high, while a linearly decreasing loss would suggest that the learning rate may be too low. A large gap between training and validation accuracy would suggest overfitting due to a large model without much regularization. No gap between training and validation accuracy would indicate low model capacity. 

You will compare networks of two and three layers using the different optimization methods you implemented. 

The different hyperparameters you can experiment with are:
- **Batch size**: We recommend you leave this at 200 initially which is the batch size we used. 
- **Number of iterations**: You can gain an intuition for how many iterations to run by checking when the validation accuracy plateaus in your train/val accuracy graph.
- **Initialization** Weight initialization is very important for neural networks. We used the initialization `W = np.random.randn(n) / sqrt(n)` where `n` is the input dimension for layer corresponding to `W`. We recommend you stick with the given initializations, but you may explore modifying these. Typical initialization practices: http://cs231n.github.io/neural-networks-2/#init
- **Learning rate**: Generally from around 1e-4 to 1e-1 is a good range to explore according to our implementation.
- **Learning rate decay**: We recommend a 0.95 decay to start.
- **Hidden layer size**: You should explore up to around 120 units per layer. For three-layer network, we fixed the two hidden layers to be the same size when obtaining the target numbers. However, you may experiment with having different size hidden layers.
- **Regularization coefficient**: We recommend trying values in the range 0 to 0.1. 

Hints:
- After getting a sense of the parameters by trying a few values yourself, you will likely want to write a few for-loops to traverse over a set of hyperparameters.
- If you find that your train loss is decreasing, but your train and val accuracy start to decrease rather than increase, your model likely started minimizing the regularization term. To prevent this you will need to decrease the regularization coefficient. 

## Hyperparameter tuning
In the output shown above, in each cell, we detail the hyperparameters generated by a coarse, first-pass search. We then carry out a finer sweep of the hyperparameter space. The code for implementing this parameter sweep is not shown in the output for the sake of brevity, but can be found in the attached notebook. We only show the client (with its results cleared, for brevity) and how to call the sweep. The higher-level details of parameter search can be found in the report instead.

In [ ]:
import psweep as ps
import pandas as pd
from typing import Union


def get_calc_dir(num_layers: int, algorithm_cls: Algorithm):
    return "calc_" + algorithm_cls.__name__ + "_{}layer".format(num_layers)

def run_campaign(params_cls, algorithm_cls, raw_parameters : List[Dict]):
    pbar = tqdm(total = len(raw_parameters), position = 1)
    
    def _run_training(pset):
        params = params_cls(hidden_size = pset['hidden_size'], 
                            epochs = pset['epochs'], 
                            learning_rate = pset['learning_rate'], 
                            learning_rate_decay = pset['learning_rate_decay'], 
                            regularization = pset['regularization'])
        pbar.update(1)
        
        # dont ensemble
        total_ensembles = 1
        stats_ensemble_history = [None for _ in range(total_ensembles)]
        test_accuracy_history = []
        for trial in range(total_ensembles):
            trained_net, stats = train_model(params, algorithm_cls, verbose = False)
            stats_ensemble_history[trial] = stats
            # _, test_accuracy[trial] = generate_report(trained_net, stats, verbose=False)
        
        return dict({'stats_history' : stats_ensemble_history}, **asdict(params))
    
    return ps.run(_run_training, raw_parameters, calc_dir = get_calc_dir(params_cls.num_layers, algorithm_cls), simulate=False)

# should not be none though
def report_results_of(
    run_df: Union[pd.DataFrame, None],
    params_cls: TrainingParams,
    algorithm_cls: Algorithm,
):
    if not isinstance(run_df, pd.DataFrame):
        import os

        run_df = ps.df_read(
            os.path.join(
                get_calc_dir(params_cls.num_layers, algorithm_cls), 'results.pk'
            )
        )

    """
    best_max_loc = None
    best_last_loc = None
    best_max = -1.0
    best_last = -1.0

    for loc_stat_h, stat_h in enumerate(run_df.stats_history):
        loc_max = -1.0
        loc_last = -1.0
        for trials in stat_h:
            loc_max = max(loc_max, trials.val_accuracy.max())
            loc_last = max(loc_last, trials.val_accuracy[-1])
        if loc_max >= best_max:
            best_max_loc = loc_stat_h
            best_max = loc_max
        if loc_last >= best_last:
            best_last_loc = loc_stat_h
            best_last = loc_last

    print("Best max validation accuracy :", best_max, " Best last validation accuracy :", best_last)
    print("Best max validation accuracy index :", best_max_loc, " Best last validation accuracy index :", best_last_loc)
    """
    # assume single trial
    last_accuracies = np.array([trial.val_accuracy[-1] for stat in run_df.stats_history for trial in stat])
    max_accuracies = np.array([trial.val_accuracy.max() for stat in run_df.stats_history for trial in stat])
    
    # Plot the loss function and train / validation accuracies
    plt.subplot(2, 1, 1)
    plt.plot(last_accuracies)
    plt.title('last_acc')
    plt.xlabel('Iteration')
    plt.ylabel('Acc')

    plt.subplot(2, 1, 2)
    plt.plot(max_accuracies)
    plt.title('max acc')
    plt.xlabel('Iteration')
    plt.ylabel('Acc')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    threshold = 0.52
    last_indices = np.where(last_accuracies > threshold)
    max_indices = np.where(max_accuracies > threshold)
    print("Best max validation accuracy :", max_accuracies[max_indices], " Best last validation accuracy :", last_accuracies[last_indices])
    print("Best max validation accuracy index :", max_indices, " Best last validation accuracy index :", last_indices)

    
    return run_df

In [ ]:
## Test of parameter sweeps
if False:
    model_cls = TwoLayerTrainingParams
    algorithm_cls = Algorithm.SGD

    sweep_hidden_size = ps.plist('hidden_size', [20])
    sweep_epochs = ps.plist('epochs', [5])
    sweep_learning_rate = ps.plist('learning_rate', [1e-2])
    sweep_learning_rate_decay = ps.plist('learning_rate_decay', [0.95])
    sweep_regularization = ps.plist('regularization', [0.005])

    df = run_campaign(
            model_cls, algorithm_cls,
            ps.pgrid(sweep_hidden_size, sweep_epochs, sweep_learning_rate, sweep_learning_rate_decay, sweep_regularization),
        )

In [ ]:
# Number of layers (Two or Three)
model_cls = ThreeLayerTrainingParams
# Algorithm
algorithm_cls = Algorithm.Adam

# Sweep parameters
sweep_hidden_size = ps.plist('hidden_size', [80, 100, 120])
sweep_epochs = ps.plist('epochs', [30])
sweep_learning_rate = ps.plist('learning_rate', [1e-2, 2e-2, 5e-2, (8e-2)])
sweep_learning_rate_decay = ps.plist('learning_rate_decay', [0.95, 0.99])
sweep_regularization = ps.plist('regularization', [0.005, 0.01, 0.02, (0.04)])

# Store as a database file
df = report_results_of(
    run_campaign(
        model_cls, algorithm_cls,
        ps.pgrid(sweep_hidden_size, sweep_epochs, sweep_learning_rate, sweep_learning_rate_decay, sweep_regularization),
    ),
    model_cls, algorithm_cls
)

In [ ]:
# Number of layers (Two or Three)
model_cls = TwoLayerTrainingParams
# Algorithm
algorithm_cls = Algorithm.Adam
df = report_results_of(None, model_cls, algorithm_cls)
df.iloc[32]

In [ ]:
plot_stats(df.iloc[32]['stats_history'][0])

## Run on the test set
When you are done experimenting, you should evaluate your final trained networks on the test set.

In [ ]:
# best_2layer_sgd_prediction = None
# best_3layer_sgd_prediction = None
# best_2layer_adam_prediction = None
# best_3layer_adam_prediction = None

## Kaggle output

Once you are satisfied with your solution and test accuracy, output a file to submit your test set predictions to the Kaggle for Assignment 2 Neural Network. Use the following code to do so:

In [ ]:
# output_submission_csv('./nn_2layer_sgd_submission.csv', best_2layer_sgd_prediction)
# output_submission_csv('./nn_3layer_sgd_submission.csv', best_2layer_sgd_prediction)
# output_submission_csv('./nn_2layer_adam_submission.csv', best_2layer_sgd_prediction)
# output_submission_csv('./nn_3layer_adam_submission.csv', best_2layer_sgd_prediction)

## Compare SGD and Adam
Create graphs to compare training loss and validation accuracy between SGD and Adam. The code is similar to the above code, but instead of comparing train and validation, we are comparing SGD and Adam.


We compare here the best parameter sets over our search space for SGD and Adam, for two and three layers respectively.

## Comparison for two layers

In [ ]:
# 2 Layer SGD
sgd_two_layer_optimal_params = {
    "hidden_size": 120,
    "epochs": 30,
    "learning_rate": 8e-2,
    "learning_rate_decay": 0.95,
    "regularization": 0.01,
}

params = TwoLayerTrainingParams(**sgd_two_layer_optimal_params)
trained_net_sgd_two_layer, stats_sgd_two_layer = train_model(params, Algorithm.SGD)


adam_two_layer_optimal_params = {
    "hidden_size": 100,
    "epochs": 30,
    "learning_rate": 1e-2,
    "learning_rate_decay": 0.95,
    "regularization": 0.005,
}
params = TwoLayerTrainingParams(**adam_two_layer_optimal_params)
trained_net_adam_two_layer, stats_adam_two_layer = train_model(params, Algorithm.Adam)

In [ ]:
def plot_stats_for_comparison(sgd_stats, adam_stats, output = False):
    import plotly.graph_objects as go
    from plotly.io import write_image
    from plotly.subplots import make_subplots

    # fig = go.Figure()
    fig = make_subplots(rows=3, 
                        cols=1, 
                        subplot_titles=("Weight norm history", "Loss history", "Accuracy history"))
    # l2 norm
    fig.add_trace(go.Scatter(y=sgd_stats.weight_norms,
                        mode='lines+markers', name="SGD W norm"), row=1, col=1)
    fig.add_trace(go.Scatter(y=adam_stats.weight_norms,
                        mode='lines+markers', name="Adam W norm"), row=1, col=1)
    fig.update_yaxes(title_text="W norm", row=1, col=1)

    # losses
    fig.add_trace(go.Scatter(y = sgd_stats.train_loss,
                        mode='lines+markers',
                        name='SGD training_loss'), row=2, col=1)
    fig.add_trace(go.Scatter(y = adam_stats.train_loss,
                        mode='lines+markers',
                        name='Adam training_loss'), row=2, col=1)
    # Update xaxis properties
    fig.update_yaxes(title_text="Loss", row=2, col=1)

    # losses
    fig.add_trace(go.Scatter(y = sgd_stats.train_accuracy,
                        mode='lines+markers',
                        name='SGD train_accuracy'), row=3, col=1)
    fig.add_trace(go.Scatter(y = sgd_stats.val_accuracy,
                        mode='lines+markers',
                        name='SGD val_accuracy'), row=3, col=1)
    fig.add_trace(go.Scatter(y = adam_stats.train_accuracy,
                        mode='lines+markers',
                        name='Adam train_accuracy'), row=3, col=1)
    fig.add_trace(go.Scatter(y = adam_stats.val_accuracy,
                        mode='lines+markers',
                        name='Adam val_accuracy'), row=3, col=1)
    # Update xaxis properties
    fig.update_yaxes(title_text="Accuracy", row=3, col=1)
    fig.update_xaxes(title_text="epochs", row=3, col=1)

    fig.show(renderer="notebook+pdf")
    if output:
        write_image(fig, 'output_file.pdf', format='pdf')

In [ ]:
plot_stats_for_comparison(stats_sgd_two_layer, stats_adam_two_layer)

## Comparison for three layers

In [ ]:
sgd_three_layer_optimal_params = {
    "hidden_size": 120,
    "epochs": 30,
    "learning_rate": 5e-2,
    "learning_rate_decay": 0.95,
    "regularization": 0.01,
}

params = ThreeLayerTrainingParams(**sgd_three_layer_optimal_params)
trained_net_sgd_three_layer, stats_sgd_three_layer = train_model(params, Algorithm.SGD)

adam_three_layer_optimal_params = {
    "hidden_size": 100,
    "epochs": 30,
    "learning_rate": 1e-2,
    "learning_rate_decay": 0.95,
    "regularization": 0.005,
}

# These are the best parameter sets found after running a fine search of the phase space
params = ThreeLayerTrainingParams(**adam_three_layer_optimal_params)
trained_net_adam_three_layer, stats_adam_three_layer = train_model(params, Algorithm.Adam)

In [ ]:
plot_stats_for_comparison(stats_sgd_three_layer, stats_adam_three_layer)